<a href="https://colab.research.google.com/github/Nerujan/Colab/blob/main/XLM_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets seqeval torch

# Import necessary libraries
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
from datasets import load_dataset, Dataset, DatasetDict
import numpy as np
from seqeval.metrics import classification_report

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.7 MB/s eta 0:00:00
 

In [ ]:
# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
model = AutoModelForTokenClassification.from_pretrained("xlm-roberta-base", num_labels=9)  # Adjust num_labels as per your dataset

# Define label mapping
label_list = ["O", "B-ORG", "I-ORG", "B-MISC", "I-MISC", "B-COMM", "I-COMM","B-PER", "I-PER"]  # Adjust as needed
label_map = {label: i for i, label in enumerate(label_list)}

# Load your dataset from Google Drive
dataset_path = "/content/drive/MyDrive/Colab Notebooks/Dataset/Final_Tamil.txt"  # Replace with actual path
dataset = load_dataset("text", data_files=dataset_path)

print(label_map)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Generating train split: 0 examples [00:00, ? examples/s]

{'O': 0, 'B-ORG': 1, 'I-ORG': 2, 'B-MISC': 3, 'I-MISC': 4, 'B-COMM': 5, 'I-COMM': 6, 'B-PER': 7, 'I-PER': 8}


In [ ]:
# Ensure that each entry in 'tokens' is a list of tokens
def parse_text(example):
    words, labels = [], []
    text_data = example.get("text", "").strip()
    if not text_data:
        return {"tokens": [], "ner_tags": []}

    for word_label in text_data.split(" "):
        parts = word_label.rsplit(" ", 1)
        if len(parts) == 2 and parts[1] in label_map:
            word, label = parts
        else:
            word, label = word_label, "O"
        words.append(word)
        labels.append(label_map.get(label, 0))  # Default to "O" if not found

    return {"tokens": words, "ner_tags": labels}


In [ ]:
def tokenize_and_align_labels(example):
    # Ensure truncation and padding are activated for the tokenizer
    tokenized_inputs = tokenizer(example["tokens"], truncation=True, padding=True, is_split_into_words=True)

    # Retrieve word ids for token alignment
    word_ids = tokenized_inputs.word_ids()
    previous_word_idx = None
    labels = []

    for word_idx in word_ids:
        if word_idx is None:
            labels.append(-100)  # Ignore special tokens
        elif word_idx != previous_word_idx:
            labels.append(example["ner_tags"][word_idx])  # Align the label with the word
        else:
            labels.append(example["ner_tags"][word_idx])  # Assign label to subword tokens
        previous_word_idx = word_idx

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


In [ ]:
def tokenize_dataset(dataset):
    tokenized_data = []
    for example in dataset:
        tokenized_data.append(tokenize_and_align_labels(example))
    return tokenized_data

# Tokenize without batch processing (one by one)
tokenized_datasets = tokenize_dataset(dataset['train'])


In [ ]:
tokenized_datasets = dataset.map(tokenize_and_align_labels)


Map:   0%|          | 0/69699 [00:00<?, ? examples/s]

Map:   0%|          | 0/17425 [00:00<?, ? examples/s]

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./results", per_device_train_batch_size=8, per_device_eval_batch_size=8,
    num_train_epochs=3, weight_decay=0.01, evaluation_strategy="epoch",
    save_strategy="epoch", logging_dir="./logs", remove_unused_columns=False
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    true_predictions = [[label_list[p] for p in pred] for pred in predictions]
    true_labels = [[label_list[l] for l in label] for label in labels]
    return classification_report(true_labels, true_predictions, output_dict=True)

In [ ]:
if "train" not in dataset or "eval" not in dataset:
    # dataset is already a DatasetDict, access the 'train' split to apply train_test_split
    train_test_dataset = dataset['train'].train_test_split(test_size=0.2)

    # Create a new DatasetDict with the split data
    dataset = DatasetDict({
        "train": train_test_dataset["train"],
        "eval": train_test_dataset["test"]
    })

print(dataset)  # Verify split worked

DatasetDict({
    train: Dataset({
        features: ['text', 'tokens', 'ner_tags'],
        num_rows: 69699
    })
    eval: Dataset({
        features: ['text', 'tokens', 'ner_tags'],
        num_rows: 17425
    })
})


In [ ]:
# ... previous code ...

# Apply tokenization AFTER creating train/eval splits
if "train" not in dataset or "eval" not in dataset:
    # dataset is already a DatasetDict, access the 'train' split to apply train_test_split
    train_test_dataset = dataset['train'].train_test_split(test_size=0.2)

    # Create a new DatasetDict with the split data
    dataset = DatasetDict({
        "train": train_test_dataset["train"],
        "eval": train_test_dataset["test"]
    })

# Apply tokenization to the updated dataset with train/eval splits
tokenized_datasets = dataset.map(tokenize_and_align_labels)

# ... rest of the code ...

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["eval"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


<ipython-input-97-094fffb28910>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`text` in this case) have excessive nesting (inputs type `list` where type `int` is expected).